In [ ]:
%load_ext autoreload
%load_ext autotime

: 

In [2]:
import os
import sys
import time
import warnings
from bokeh.io import output_notebook

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

# Set the notebook to display Bokeh plots
output_notebook()

Loading BokehJS ...

time: 1.19 s (started: 2024-11-10 17:04:54 -08:00)


In [3]:
analyzer = "dftracer"

run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"

cluster = "external"
cluster_restart_on_connect = True
cluster_scheduler_address = "tcp://127.0.0.1:46185"

checkpoint = False
exclude_characteristics = ['node_count', 'app_count', 'proc_count', 'file_count']
logical_view_types = False
percentile = 0.95
time_granularity = 1e6
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

app_metrics = [
    'io_compute_ratio',
    'io_time',
    'ops',
    'u_checkpoint_io_time',
    'u_io_time',
    'u_read_io_time',
]
dataloader_metrics = ['time', 'iops']
posix_metrics = ['time', 'iops']

app_view_types = ['proc_name', 'time_range']
dataloader_view_types = ['proc_name', 'time_range']
posix_view_types = ['proc_name', 'time_range']

time_view_type = 'time_range'
unoverlapped_posix_only = False

output_compact = False
output_group_behavior = False
output_max_bottlenecks = 1
output_root_only = True


time: 1.49 ms (started: 2024-11-10 17:05:00 -08:00)


In [ ]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.cluster import ExternalCluster
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"analyzer.time_granularity={time_granularity}",
            # f"cluster={cluster}",
            # f"cluster.restart_on_connect={cluster_restart_on_connect}",
            # f"cluster.scheduler_address={cluster_scheduler_address}",
            f"exclude_characteristics=[{','.join(exclude_characteristics)}]",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.compact={output_compact}",
            f"output.group_behavior={output_group_behavior}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    if isinstance(cluster, ExternalCluster):
        client = Client(cluster.scheduler_address)
        if cluster.restart_on_connect:
            client.restart()
    else:
        client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 26.6 s (started: 2024-11-10 17:05:24 -08:00)


In [5]:
client.dashboard_link

'http://127.0.0.1:8787/status'

time: 6.2 ms (started: 2024-11-10 17:05:56 -08:00)


In [ ]:
from wisio.constants import Layer

result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    bottleneck_rules=cfg.bottleneck_rules,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    percentile=cfg.percentile,
    metrics={
        Layer.APP: app_metrics,
        Layer.DATALOADER: dataloader_metrics,
        Layer.POSIX: posix_metrics,
    },
    time_view_type=time_view_type,
    unoverlapped_posix_only=unoverlapped_posix_only,
    view_types={
        Layer.APP: app_view_types,
        Layer.DATALOADER: dataloader_view_types,
        Layer.POSIX: posix_view_types,
    },
)

time: 11.3 s (started: 2024-10-14 23:41:41 -07:00)


In [ ]:
analyzer.write_bottlenecks(result.flat_bottlenecks)

In [ ]:
output.handle_result(metrics=[], result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          145.36 seconds                                                                                │
│  I/O Time         1.35 seconds                                                                                  │
│                   ├── Read - 0.03 seconds (1.96%)                                                               │
│                   ├── Write - 0.15 seconds (10.86%)                                                             │
│                   └── Metadata - 1.19 seconds (88.42%)                                                          │
│  I/O Operations   231,337 ops                                                                                   │
│                   ├── Read - 13,489 ops (5.83%)                                                                 │
│                   ├── Write - 112,353 ops (48.57%)                                                              │
│                   └── Metadata - 105,063 ops (45.42%)                                                           │
│  I/O Size         690.06 MiB                                                                                    │
│                   ├── Read - 1.76 MiB (0.26%)                                                                   │
│                   └── Write - 688.30 MiB (99.74%)                                                               │
│  Read Requests    4 kiB-16 kiB - 13,489 ops                                                                     │
│                   ├── <4 kiB - 13,440 ops (99.64%)                                                              │
│                   └── 4-16 kiB - 49 ops (0.36%)                                                                 │
│  Write Requests   4 kiB-64 kiB - 112,353 ops                                                                    │
│                   ├── <4 kiB - 1,945 ops (1.73%)                                                                │
│                   ├── 4-16 kiB - 107,192 ops (95.41%)                                                           │
│                   └── 16-64 kiB - 3,216 ops (2.86%)                                                             │
│  Nodes            1 node                                                                                        │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  48 processes                                                                                  │
│  Files            2,556 files                                                                                   │
│                   ├── Shared: 669 files (26.17%)                                                                │
│                   └── FPP: 1,887 files (73.83%)                                                                 │
│  Time Periods     126 time periods (Time Granularity: 1,000,000.0)                                              │
│  Access Pattern   Sequential: 125,842 ops (100.00%) - Random: 0 ops (0.00%)                                     │
│                                                                                                                 │
╰─ R: Read - W: Write - M: Metadata  ─────────────────────────────────────────────────────────────────────────────╯
                                                                                                                   
╭─────────────────────────────────── I/O Time: 1 I/O Bottleneck with 3 Reasons ───────────────────────────────────╮
│                                                                                                                 │
│  Process View (1 bottleneck with 3 reasons)           

time: 2.35 s (started: 2024-10-13 22:58:14 -07:00)


In [ ]:
bottlenecks = analyzer.read_bottlenecks()

In [ ]:
from bokeh.io import show
from wisio.plots import plot_bottleneck_summary

show(plot_bottleneck_summary(bottlenecks, groupby=('layer', 'view_name')))

76778

time: 255 ms (started: 2024-10-14 23:42:38 -07:00)


In [ ]:
show(plot_bottleneck_summary(bottlenecks, groupby=('layer', 'view_name', 'metric')))